# Requirements

In [ ]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.


## Import

In [2]:
# aut-reloader
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo

from nn import NeuralNetwork, Trainer
from utils.losses import Loss
from utils.data_manage import DataLoader, StandardScaler


ModuleNotFoundError: No module named 'numpy'

In [ ]:
# fetch dataset
monk_s_problems = fetch_ucirepo(id=70)

# data (as pandas dataframes)
X = monk_s_problems.data.features
y = monk_s_problems.data.targets

# metadata
print(monk_s_problems.metadata)

# variable information
print(monk_s_problems.variables)

In [ ]:
#one hot encode of X
X = np.array(X)
X_onehot = []
for column in range(X.shape[1]):
  X[:, column] = X[:, column] - 1 #needed to use np.eye
  n_unique = len(np.unique(X[:, column]))
  X_onehot.append(np.eye(n_unique)[X[:, column]])
X_onehot = np.concatenate(X_onehot, axis=1)


monk_dataset = DataLoader(X_dataset= np.array(X_onehot), y_dataset= np.array(y))
X_train, y_train, X_val, y_val = monk_dataset.train_val_split(portion = 0.75)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
NEURAL_NETWORK_CONFIGURATIONS = [([17, 8, 1],['tanh', 'sigmoid']),
                                 ([17, 8, 1],['leaky relu', 'sigmoid']),
                                 ([17, 8, 8, 1],['tanh', 'tanh', 'sigmoid']),
                                 ([17, 8, 8, 1],['leaky relu', 'leaky relu', 'sigmoid']),
                                 ([17, 16, 1],['tanh', 'sigmoid']),
                                 ([17, 16, 1],['leaky relu', 'sigmoid']),
                                 ([17, 16, 16, 1],['tanh', 'tanh', 'sigmoid']),
                                 ([17, 16, 16, 1],['leaky relu', 'leaky relu', 'sigmoid']),
                                 ]
ETA_CONFIGURATIONS = [0.01, 0.001, 0.0001]
LAMBDA_CONFIGURATIONS = [0, 1e-3, 1e-5]
ALPHA_CONFIGURATIONS = [0, 0.5, 0.9]
EPOCHS = 300
LOSS_FUNCTION = Loss('mse')

for NEURAL_NETWORK_ARCHITECTURE, NEURAL_NETWORK_ACTIVATION in NEURAL_NETWORK_CONFIGURATIONS:
  for ETA in ETA_CONFIGURATIONS:
    for LAMBDA in LAMBDA_CONFIGURATIONS:
      for ALPHA in ALPHA_CONFIGURATIONS:
        print(f"NEURAL_NETWORK_ARCHITECTURE: {NEURAL_NETWORK_ARCHITECTURE}, NEURAL_NETWORK_ACTIVATION: {NEURAL_NETWORK_ACTIVATION}, ETA: {ETA}, LAMBDA: {LAMBDA}, ALPHA: {ALPHA}")
        #train
        nn = NeuralNetwork(NEURAL_NETWORK_ARCHITECTURE, NEURAL_NETWORK_ACTIVATION, std=0.2)
        trainer = Trainer(
            nn=nn,
            loss=LOSS_FUNCTION,
            X_train=X_train,
            y_train=y_train, #no scaling y because of onehot
            X_val=X_val,
            y_val=y_val,
            epochs=EPOCHS,
            early_stopping=50, # no improvements in 50 epochs_> stop
            eta=ETA,               # Learning rate iniziale
            lam=LAMBDA,                # L2
            alpha=ALPHA,               # Momentum
            tau=150,              # Decay linear
            std_scale_x=False,       # we scaled before
            std_scale_y=False,      #no on onehot
            batch_size=16,
            shuffle_batches=True
        )

        # return_best_nn=True returns the best nn
        best_nn = trainer.train(return_best_nn=True, print_epochs=False, plot_epochs=True)

        #best val accuracy
        out = best_nn.forward(X_val)[-1][-1]
        predictions = np.round(out)
        print(f"Accuracy: {np.mean(predictions == y_val) * 100}%")